In [ ]:
#importing all the libraries
import numpy as np
import pandas as pd

In [ ]:
#Take the main data set
movie=pd.read_csv("movies_metadata.csv")
movie.head()
movie.shape

In [ ]:
movie=movie.drop([19730, 29503, 35587]) # Dropping the bad lines 
movie=movie[['id','original_title']]    #Taking subset of the main dataset
movie.columns=['tmdbId','movie_name']   #giving the column name...
movie['tmdbId']=movie['tmdbId'].astype('float')  
movie.info()

In [ ]:
link=pd.read_csv("links.csv")# take the link dataset...
link.info()
len(link)

In [26]:
rating=pd.read_csv("ratings_small.csv")#take final dataset
rating['movieId'].value_counts().head(3)
len(rating)

100004

In [27]:
Id_min=rating['movieId'].value_counts()#those id atleast gave rating on a particular movieId
len(Id_min)
Id_min.head(5)

356    341
296    324
318    311
593    304
260    291
Name: movieId, dtype: int64

In [28]:
#we are taking only those id's who has given minimum 35 rating
Id_min=Id_min[Id_min>35]
len(Id_min)

696

In [29]:
ratings=rating[rating['movieId'].isin(Id_min.index)]
len(ratings)

53326

In [30]:

rating_with_link=ratings.merge(link,on='movieId') #jst merge on movieId
rating_with_link.head(2)
rating_with_link.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 53326 entries, 0 to 53325
Data columns (total 6 columns):
userId       53326 non-null int64
movieId      53326 non-null int64
rating       53326 non-null float64
timestamp    53326 non-null int64
imdbId       53326 non-null int64
tmdbId       53281 non-null float64
dtypes: float64(2), int64(4)
memory usage: 2.8 MB


In [31]:
movie_final=movie.merge(rating_with_link,on='tmdbId')
movie_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53281 entries, 0 to 53280
Data columns (total 7 columns):
tmdbId        53281 non-null float64
movie_name    53281 non-null object
userId        53281 non-null int64
movieId       53281 non-null int64
rating        53281 non-null float64
timestamp     53281 non-null int64
imdbId        53281 non-null int64
dtypes: float64(2), int64(4), object(1)
memory usage: 3.3+ MB


In [32]:
movie_final1=movie_final.groupby('movie_name')['rating'].agg('count').reset_index().rename(columns={'rating':'num_ratings'})
movie_final1.head()

,movie_name,num_ratings
0,(500) Days of Summer,45
1,10 Things I Hate About You,57
2,101 Dalmatians,37
3,12 Angry Men,74
4,"20,000 Leagues Under the Sea",37


In [33]:
ultimate_final=movie_final.merge(movie_final1,on='movie_name')
ultimate_final.head()

,tmdbId,movie_name,userId,movieId,rating,timestamp,imdbId,num_ratings
0,862.0,Toy Story,7,1,3.0,851866703,114709,247
1,862.0,Toy Story,9,1,4.0,938629179,114709,247
2,862.0,Toy Story,13,1,5.0,1331380058,114709,247
3,862.0,Toy Story,15,1,2.0,997938310,114709,247
4,862.0,Toy Story,19,1,3.0,855190091,114709,247


In [34]:
ultimate_final=ultimate_final[ultimate_final['num_ratings']>=50]
len(ultimate_final)

43127

In [35]:
ultimate_final=ultimate_final.drop_duplicates(['userId','movie_name'])
len(ultimate_final) #duplicates colmn short out

43108

In [36]:
ultimate_pivot=ultimate_final.pivot_table(values='rating',index='movie_name',columns='userId').fillna(0)


In [37]:
ultimate_pivot.shape #pivot ka shape 

(453, 670)

In [38]:
from scipy.sparse import csr_matrix
ultimate_sparse=csr_matrix(ultimate_pivot)

In [39]:
from sklearn.neighbors import NearestNeighbors
nn=NearestNeighbors(metric='cosine',algorithm='brute')
nn.fit(ultimate_sparse)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)

In [40]:

    movie_index=np.where(ultimate_pivot.index==name)[0][0]
    distances,suggestions=nn.kneighbors(ultimate_pivot.iloc[movie_index,:].values.reshape(1,-1),n_neighbors=7)
    suggestions=suggestions.ravel()
    suggestions
    for i  in range(len(suggestions)):
        print(ultimate_pivot.index[suggestions[i]]) 

IndentationError: unexpected indent (<ipython-input-40-6f6ffbee6593>, line 2)

In [ ]:
user_input=input('enter a movie name :')
suggest(user_input)